# 并行编程


## 并行编程模型
- 多线程模型
- 消息传递模型
    - 范式1：Actor/相互独立
        - Actors Mailbox
    - 范式2：CSP（GO语言使用该范式）
- 数据并行模型
    - [Parameter server distributed for machine learning](https://pdfs.semanticscholar.org/30e9/4e24d67994c5a8e2f20f852a51d28a720de2.pdf)
    - 每个结点均有一份模型
    - 各个结点取不同的数据，batch_size
    - 各个模型梯度下降，返回计算的参数给参数服务器
    - 参数服务器更新参数
- Task farming(Master-Worker)

## 基于线程的并行
- 线程是进程的组件，一个进程可以有多个线程
- 多个线程共享父进程的内存空间
- 由OS调度
- 伪多线程-GIL

In [33]:
from threading import Thread
import threading

In [24]:
def my_count():
    i = 0
    for _ in range(10000000):
        i += 1
    return 

In [25]:
def single_thread():
    t = Thread(target=my_count)
    t.start()
    t.join()

In [28]:
%%time
single_thread()
single_thread()

Wall time: 1.4 s


In [45]:
def multi_thread():
    threads = []
    for i in range(2):
        t = Thread(target=my_count)
        t.start()
        threads.append(t)
    for t in threads:
        t.join()

In [46]:
%%time
multi_thread()

Wall time: 1.37 s


In [49]:
def f():
    print("Function called by thread ", threading.current_thread().name)
    for i in range(8):
        print(f"thread {threading.current_thread().name} >>> {i}")

    print(f"thread {threading.current_thread().name} ended")

In [51]:
print(f'thread {threading.current_thread().name} is running...')

t = threading.Thread(target=f, name='LoopThread')
t.start()
t.join()

print(f'thread {threading.current_thread().name} ended.')

thread MainThread is running...
Function called by thread  LoopThread
thread LoopThread >>> 0
thread LoopThread >>> 1
thread LoopThread >>> 2
thread LoopThread >>> 3
thread LoopThread >>> 4
thread LoopThread >>> 5
thread LoopThread >>> 6
thread LoopThread >>> 7
thread LoopThread ended
thread MainThread ended.


In [57]:
print(f'thread {threading.current_thread().name} is running...')

t = threading.Thread(target=f, name='LoopThread')
t.start()
# t.join()

print(f'thread {threading.current_thread().name} ended.')

thread MainThread is running...
Function called by thread  LoopThread
thread LoopThread >>> 0
thread LoopThread >>> 1
thread LoopThread >>> 2
thread LoopThread >>> 3
thread LoopThread >>> 4
thread LoopThread >>> 5
thread LoopThread >>> 6
thread LoopThread >>> 7thread MainThread ended.

thread LoopThread ended


## 基于进程的并行
- 拥有独立内存空间
- 充分利用多核
- 进程切换开销大

In [1]:
import multiprocessing

In [2]:
def f2(name):
    print(f'Thread {multiprocessing.current_process().name}')
    print(f'Hello {name}')

In [3]:
print(f'Thread {multiprocessing.current_process().name}')

Thread MainProcess


In [4]:
p = multiprocessing.Process(target=f2, args=('rambo',))
p.start()
p.join()

In [5]:
multiprocessing.cpu_count()

8

In [6]:
def f1(x):
    return x*2

In [ ]:
# something wrong with these code under jupyter process
# ok under pycharm envs.
with multiprocessing.Pool(multiprocessing.cpu_count()) as p:
    print(p.map(f1, list(range(10))))

## 同步
- 进程和线程均涉及资源抢占的问题，如：银行账户问题

In [13]:
class Account:
    def __init__(self, money):
        self._money = money
    
    def add(self, money):
        self._money += money
    
    def subs(self, money):
        self._money -= money
    
    @property
    def balance(self):
        return self._money

In [71]:
def run(money):
    for _ in range(1000000):
        account.add(money)
        account.subs(money)

In [72]:
# 没有线程锁的情况下出现同步的问题
account = Account(1000)
print(account.balance)
t1 = Thread(target=run, args=(100, ))
t2 = Thread(target=run, args=(200,))
t1.start()
t2.start()
t1.join()
t2.join()
account.balance

1000


900

In [49]:
# 加锁改善
from threading import Lock

In [55]:
class AccountLock:
    def __init__(self, money):
        self._money = money
        self._lock = Lock()
    
    def add(self, money):
        with self._lock:
            self._money += money
    
    def subs(self, money):
        with self._lock:
            self._money -= money
    
    @property
    def balance(self):
        return self._money

In [74]:
# 加锁后
account = AccountLock(1000)
print(account.balance)
t1 = Thread(target=run, args=(100, ))
t2 = Thread(target=run, args=(200,))
t1.start()
t2.start()
t1.join()
t2.join()
account.balance

1000


1000

## 死锁问题

# 分布式

## 分布式理论
- CAP（三选二）
    - Consistency 一致性
    - Available 可用性
    - Partition-Tolerance 容错性

## 使用manager实现简单的分布式计算

## Celery/Ray分布式框架

## 分布式实战